# Multiplicación de Matrices utilizando CUDA.jl y KernelAbstractions.jl

Realizaremos un programa que ejecute la multiplicación de matrices de manera paralela usando las bibliotecas `CUDA.jl` y `KernelAbstractions` para comparar su funcionamiento, implementación y resultados.


In [16]:
using KernelAbstractions, CUDA, CUDAKernels, Test


## KernelAbstractions

Comenzamos definiendo el kernel usando el macro `@kernel` y definiendo la lógica de nuestro programa.

In [26]:
@kernel function matmul_kernel_ka!(a, b, c)
    #Obtenemos los índices globales en forma de tupla
    i, j = @index(Global, NTuple)

    #variable temporal para hacer la suma de los índices
    
    tmp_sum = zero(eltype(c))

    #obtenemos el resultado que irá en la posición c[i,j] 
    for k = 1:size(a)[2]
        tmp_sum += a[i,k] * b[k,j]
    end

    #asignamos el resultado
    c[i,j] = tmp_sum
end

Vamos a crear una función para invocar el kernel y evitar que se lance con errores.

In [22]:
function matmul_ka!(a,b,c)
    if size(a)[2] != size(b)[1]
        println("El tamaño de las matrices no coincide!")
        return nothing
    end
    if isa(a, Array)
        kernel! = matmul_kernel_ka!(CPU(),4)
    else
        kernel! = matmul_kernel_ka!(CUDADevice(), 256)
    end
    kernel!(a,b,c,ndrange=size(c))
end

matmul_ka! (generic function with 1 method)

Para probar la ejecución del kernel, definiremos primero los 3 arreglos donde estarán nuestros operandos y el resultado. Luego se lo pasaremos a la función en forma de `CuArray` para que se ejecute dentro del GPU.

In [19]:
a = rand(256,123);
b = rand(123,45);
c = zeros(256,45);

In [ ]:

if has_cuda_gpu()
    d_a = CuArray(a)
    d_b = CuArray(b)
    d_c = CuArray(c)

    ev = matmul_ka!(d_a, d_b, d_c)
    wait(ev)

end

## CUDA.jl

Para realizar el programa en `CUDA.jl` se necesita la biblioteca `CUDA` (que ya incluimos para hacer uso de `CuArray`) y definir el kernel sin necesidad del macro `@kernel`. Así mismo, la obtención del índice se hace usando las funciones `threadIdx()`, `blockIdx()` y `blockDim()`.

In [34]:
function matmul_kernel_cu!(a,b,c)
    i = threadIdx().x + (blockIdx().x -1) * blockDim().x
    j = threadIdx().y + (blockIdx().y -1) * blockDim().y 

    tmp_sum = zero(eltype(c))

    for k = 1:size(a)[2]
        tmp_sum += a[i,k] * b[k,j]
    end

    c[i,j] = tmp_sum
    return nothing
end

matmul_kernel_cu! (generic function with 1 method)

Como se puede ver, el kernel es prácticamente igual con la única diferencia siendo la manera de obtener los índices `i` y `j`.

Para ejecutar el kernel, se usa la macro @cuda especificando el número de hilos y bloques.

In [31]:
d_a = CUDA.rand(256,256);
d_b = CUDA.rand(256,256);
d_c = CuArray(zeros(256,256));

In [36]:

function matmul_cu(a,b,c)
    if size(a)[2] != size(b)[1]
        println("El tamaño de las matrices no coincide.")
        return nothing
    end

    m,n = size(c)
    n_threads = (32,32)
    n_blocks = (div(m,32),div(m,32))
    @sync @cuda threads=n_threads blocks=n_blocks matmul_kernel_cu!(a,b,c)

end


matmul_cu (generic function with 1 method)

In [37]:
matmul_cu(d_a,d_b,d_c)

@test isapprox(Array(d_c), Array(d_a)*Array(d_b))

Test Passed